In [ ]:
import glob
import os
import librosa
import numpy as np
import tensorflow as tf
import sounddevice
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
# Boton
import RPi.GPIO as GPIO
from time import sleep
#correo
import smtplib
import RPi.GPIO as GPIO
import time
# SERVER
import os
from http.server import BaseHTTPRequestHandler, HTTPServer
def setupGPIO():
    GPIO.setmode(GPIO.BCM)
    GPIO.setwarnings(False)
    GPIO.setup(24, GPIO.OUT)
def getTemperature():
    temp = os.popen("/opt/vc/bin/vcgencmd measure_temp").read()
    return temp
class MyServer(BaseHTTPRequestHandler):
    def do_HEAD(self):
        self.send_response(200)
        self.send_header('Content-type', 'text/html')
        self.end_headers()
    def _redirect(self, path):
        self.send_response(303)
        self.send_header('Content-type', 'text/html')
        self.send_header('Location', path)
        self.end_headers()
    def do_GET(self):
        html = '''
           <html>
           <body 
            style="width:960px; margin: 20px auto;">
           <h1>Welcome to my Raspberry Pi</h1>
           <p>Current GPU temperature is {}</p>
           <form action="/" method="POST">
               Turn LED :
               <input type="submit" name="submit" value="On">
               <input type="submit" name="submit" value="Off">
           </form>
           </body>
           </html>
        '''
        temp = getTemperature()
        self.do_HEAD()
        self.wfile.write(html.format(temp[5:]).encode("utf-8"))
    def do_POST(self):
        content_length = int(self.headers['Content-Length'])
        post_data = self.rfile.read(content_length).decode("utf-8")
        post_data = post_data.split("=")[1]
        setupGPIO()
        if post_data == 'On':
            GPIO.output(24, GPIO.HIGH)
        else:
            GPIO.output(24, GPIO.LOW)
        print("LED is {}".format(post_data))
        self._redirect('/')  # Redirect back to the root url
#
class Emailer:
    def sendmail(self, recipient, subject, content):
        #Create Headers
        headers = ["From: " + GMAIL_USERNAME, "Subject: " + subject, "To: " + recipient,
                   "MIME-Version: 1.0", "Content-Type: text/html"]
        headers = "\r\n".join(headers)
        #Connect to Gmail Server
        session = smtplib.SMTP(SMTP_SERVER, SMTP_PORT)
        session.ehlo()
        session.starttls()
        session.ehlo()
        #Login to Gmail
        session.login(GMAIL_USERNAME, GMAIL_PASSWORD)
        #Send Email & Exit
        session.sendmail(GMAIL_USERNAME, recipient, headers + "\r\n\r\n" + content)
        session.quit
def extract_features():
    X = sounddevice.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, blocking=True)
    sounddevice.wait()
    X= np.squeeze(X)
    mfccs_features = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    mfccs_scaled_features = mfccs_scaled_features.reshape(1,-1)
    return mfccs_scaled_features
#
GPIO.setwarnings(False)
GPIO.setmode(GPIO.BCM)
Button = 23
LED = 24
GPIO.setup(Button, GPIO.IN, pull_up_down = GPIO.PUD_UP)
GPIO.setup(LED, GPIO.OUT)
#Email Variables
SMTP_SERVER = 'smtp.gmail.com' # Email Server (don't change!)
SMTP_PORT = 587 # Server Port (don't change!)
GMAIL_USERNAME = 'sonido.detector@gmail.com' # change this to match your gmail account
GMAIL_PASSWORD = 'Detectorsonido123'  # change this to match your gmail password
#
host_name = '192.168.8.100'  # IP Address of Raspberry Pi
host_port = 8000
#
new_model = tf.keras.models.load_model('/home/pi/Desktop/predictor.hdf5')
#new_model.summary()
filename="/home/pi/Desktop/124389-8-1-8.wav"
#filename="/home/pi/Desktop/100652-3-0-0.wav"
audio, sample_rate = librosa.load(filename, res_type='kaiser_fast') 
mfccs_features = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=40)
mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
mfccs_scaled_features=mfccs_scaled_features.reshape(1,-1)
# print(mfccs_scaled_features.shape)
predicted_label=new_model.predict_classes(mfccs_scaled_features)
print(predicted_label)
#
flag = 1
detectar_clase = 8
sender = Emailer()
#
sounddevice.default.device = 'USB PnP Sound Device'
duration = 0.1 # seconds
sample_rate = 44100
new_model = tf.keras.models.load_model('/home/pi/Desktop/predictor.hdf5')
while 1:
        mfccs_scaled_features = extract_features()
        predicted_label = new_model.predict_classes(mfccs_scaled_features)
        print(predicted_label)
        button_state = GPIO.input(Button)
        if button_state == 0:
                sleep(0.7)
                if flag == 0:
                        flag = 1
                else:
                        flag = 0
        if flag == 1 and predicted_label == detectar_clase:
                GPIO.output(LED, GPIO.HIGH)
                sendTo = 'XXXXXXXXXXX'
                emailSubject = "Button Press Detected!"
                emailContent = "yo predigo con mi modelo .- The button has been pressed at: " + time.ctime()
                sender.sendmail(sendTo, emailSubject, emailContent)
                print("Email Sent")
                sleep(0.6)
        else:
                GPIO.output(LED, GPIO.LOW)
                print('buzzer off')
# http_server = HTTPServer((host_name, host_port), MyServer)
# print("Server Starts - %s:%s" % (host_name, host_port))
# try:
#     http_server.serve_forever()
# except KeyboardInterrupt:
#     http_server.server_close()                

In [ ]:
import glob
import os
import librosa
import numpy as np
import tensorflow as tf
import sounddevice
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from tensorflow import keras
sounddevice.default.device = 'USB PnP Sound Device'
duration = 0.1 # seconds
sample_rate = 44100
new_model = tf.keras.models.load_model('/home/pi/Desktop/predictor.hdf5')

def extract_features():
    X = sounddevice.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, blocking=True)
    sounddevice.wait()
    X= np.squeeze(X)
    mfccs_features = librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40)
    mfccs_scaled_features = np.mean(mfccs_features.T,axis=0)
    mfccs_scaled_features = mfccs_scaled_features.reshape(1,-1)
    return mfccs_scaled_features
while 1:
    mfccs_scaled_features = extract_features()
    predicted_label = new_model.predict_classes(mfccs_scaled_features)
    print(predicted_label)